## Finding dishes with food in a given dining image

### Uses the following steps to identify image of food in a given image
#### 1. Pick 100 images each of Entree, Dessert, Salad and Non-food to train Scikit-Learn SVC classifier
#### 2. Pre-process all training images by first resizing to a fixed size
#### 3. Use HOG descriptor to get correct information about the image
#### 4. Extracted the colors from every image and made three new columns representing the RED, GREEN, BLUE colors. 
#### 5. Fit all of the 400 images to SVC to train the classifier
#### 6. use each of the contour to predict the trained SVC classifier

In [46]:
import os
from os import listdir
from os.path import isfile, join
from io import open
from time import time

import cv2
import numpy as np
from random import shuffle
import imutils #need to install this with pip


from skimage import data, color, exposure
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
label_list = ["Entree", "Salad", "Dessert"]

In [48]:
# Utility method to get all the images.
def get_file_list_for_label(label):
    """
    Returns the list of files for a given label. Assumes that the image documents are arranged into
    folder with same name as the labels under '../Data' directory.
    """
    data_file_path = "../Data/Training/" + label
    return [join(data_file_path, f) for f in listdir(data_file_path) if isfile(join(data_file_path, f))]

In [49]:
# Constructs and returns a list of images and the corresponding labels
# The feature at this point is nothing but the image itself. Later in pre-processing,
# the image content will be reduced into features using HOG.
def get_images_and_labels():
    """
    Returns all the image contents (features), their corresponding label names as 
    two a list of tuples.
    """
    image_label_list = []
    for label in label_list:
        file_list = get_file_list_for_label(label)
        for file in file_list:
            img = cv2.imread(file)
            image_label_list.append((img, label))
    #shuffle(image_label_list)
    return image_label_list

#### Setting up the HOG class used to make HOGS

In [50]:
minDim = 80
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9

dims = (minDim, minDim)
hog = cv2.HOGDescriptor(dims, blockSize, blockStride, cellSize, nbins)

#### Geting the HOGs from our images

In [51]:
def get_hog_histograms_with_labels(image_label_list, hog, dims):
    data = []
    for img_label in image_label_list: 
        img = cv2.resize(img_label[0], dims)
        #img = cv2.Canny(img,750,150)

        #for images with transparency layer, reduce to 3 layers
        hist = hog.compute(img)
        
        data.append((hist, img_label[1]))
    #shuffle(data)
    return map(list, zip(*data))

In [52]:
# Getting hog features for all the images, concatenate positive and negatives into an array,
image_label_list = get_images_and_labels()
X, y = get_hog_histograms_with_labels(image_label_list, hog, dims)

X = [x.flatten() for x in X]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

#### Classifier: Support Vector Machine (SVM)

In [57]:
lsvm = SVC(kernel='linear', C = 1.0, probability=True)
lsvm.fit(X_train, y_train)

acc_score = lsvm.score(X_test, y_test)
print("Support Vector Machine Classifier ", acc_score)
y_pred = lsvm.predict(X_test)
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Support Vector Machine Classifier  0.6
             precision    recall  f1-score   support

    Dessert       0.74      0.85      0.79        20
     Entree       0.35      0.33      0.34        18
      Salad       0.65      0.59      0.62        22

avg / total       0.59      0.60      0.59        60

Confusion Matrix:
 [[17  3  0]
 [ 5  6  7]
 [ 1  8 13]]


#### Gradient Boosting Classifier

In [54]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(X_train, y_train)

prediction = clf.predict(X_test)
acc_score = accuracy_score(y_test, prediction)
print("Gradient Boosting Classifier: ", acc_score)

print(classification_report(y_test, prediction))
print(confusion_matrix(y_test, prediction))

Gradient Boosting Classifier:  0.716666666667
             precision    recall  f1-score   support

    Dessert       0.83      0.95      0.88        20
     Entree       0.59      0.56      0.57        18
      Salad       0.70      0.64      0.67        22

avg / total       0.71      0.72      0.71        60

[[19  0  1]
 [ 3 10  5]
 [ 1  7 14]]


### Image detection using color processing